<a href="https://colab.research.google.com/github/HwangJae-won/2021_SEMINAR-IN-STATISTICS/blob/main/0516__Data_Basic_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Data

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/신용카드 사용자 예측/train.csv')
test=pd.read_csv('/content/drive/MyDrive/신용카드 사용자 예측/test.csv')
sample=pd.read_csv('/content/drive/MyDrive/신용카드 사용자 예측/sample_submission.csv')
train

In [ ]:
sample #형식 관련 문의해보기 
#현재 credict data 자체가 0,1,2로 구성되어 있음. 한 index마다 0,1,2 labeling 해서 나타내면 되는듯 ! (0인 경우 0열에 1로, 1,2열은 0으로 표시하는 식)

# 2. Check Data & variable

In [ ]:
train.columns

In [ ]:
train.info() 

- 연체 정도를 예측-> credit: 연체 대금 기준으로 한 신용도 
- credit가 낮으면 연체 정도가 적을것 
- 신상 정보를 바탕으로 credit 예측하기 

In [ ]:
train_y = train['credit'].astype(int)
train_y

### 1) binary variable: 변수 분포 확인

credit 변수는 0,1,2로 구성되어 있는데 2인 사용자가 훨씬 많음. 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 3))

sns.distplot(train_y[train['gender']=='M'], ax=axes[0], color="red")
sns.distplot(train_y[train['gender']=='F'], ax=axes[0], color="skyblue")

sns.distplot(train_y[train['car']=='N'], ax=axes[1], color="red")
sns.distplot(train_y[train['car']=='Y'], ax=axes[1], color="skyblue")

sns.distplot(train_y[train['reality']=='N'], ax=axes[2], color="red")
sns.distplot(train_y[train['reality']=='Y'], ax=axes[2], color="skyblue")

axes[0].set_title('gender')
axes[1].set_title('car')
axes[2].set_title('reality')


- 

- 여성이 많이 이용
- 차를 가지지 않음 사람이 더 많음
- 부동산 소유자가 더 많음 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 3))

sns.distplot(train_y[train['work_phone']==0], ax=axes[0], color="red")
sns.distplot(train_y[train['work_phone']==1], ax=axes[0], color="skyblue")

sns.distplot(train_y[train['phone']==0], ax=axes[1], color="red")
sns.distplot(train_y[train['phone']==1], ax=axes[1], color="skyblue")

sns.distplot(train_y[train['email']==0], ax=axes[2], color="red")
sns.distplot(train_y[train['email']==1], ax=axes[2], color="skyblue")

axes[0].set_title('work_phone')
axes[1].set_title('phone')
axes[2].set_title('email')

credit2의 email 여부에 따른 차이가 다른 변수보다 큼. 

In [ ]:
#0,1로 채움
train['gender'] = train['gender'].replace(['F','M'],[0,1])
train['car'] = train['car'].replace(['N','Y'],[0,1])
train['reality'] = train['reality'].replace(['N','Y'],[0,1])

### 2) 직업 유형에서 결측치 처리 방안 
- 단순 제거해도 되나?

In [ ]:
train["occyp_type"].unique() #무직에 대한 값은 없음

무직으로 볼 지 아니면 결측치 내에서도 다른 그룹으로 나눌지 : 분포보고 1~2개의 그룹으로 나눌지 생각해봐야 할 것 같음 </br>
결측치 있는 행 뽑아서 다른 변수들로 분포를 확인해보기 

In [ ]:
#결측치 있는 
df=train.copy()
df=df.fillna("NA")
df[df["occyp_type"]=="NA"]

In [ ]:
sns.distplot(df["credit"]) #credit의 비율은 전체 데이터와 비슷함

In [ ]:
df

In [ ]:
sns.violinplot(x='edu_type', y='credit', data=df)
#다 비슷한 모양 .. 굳이 나누자면 처음 3개가 credit 1의 비율이 좀 더 큼 

In [ ]:
sns.violinplot(x='income_type', y='credit', data=df)
#student의 분포만 다름. 당연함,,, 학생임

In [ ]:
sns.violinplot(x='family_type', y='credit', data=df)
#비슷한데,,,

In [ ]:
sns.violinplot(x='house_type', y='credit', data=df)
#오 그나마 다름 

In [ ]:
df['house_type'].unique()

- 묶어서 확인


## 3) 변수들간 연관도 확인

In [ ]:
train_f = train.loc[:,["child_num", "income_total","family_size", "credit"]]

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data = train_f.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

credit과는 특별한 연관 없음. 전체적으로 지금 연관도를 보는 것은 의미 없음 

더미 변수들이 많아 어떻게 처리할지가 관건
: 연관 높은 더미 변수들 조합해서 새로운 특성을 만들어보는 것은 어떨까? 

## 4) credit 변수 확인

In [ ]:
plt.subplots(figsize = (8,8))
plt.pie(train['credit'].value_counts(), labels = train['credit'].value_counts().index, 
        autopct="%.2f%%", shadow = True, startangle = 90)
plt.title('credit ratio', size=20)
plt.show()

In [ ]:
sns.displot(train["credit"])

- 연체 많이 한 사람 비율이 높은거
- 어떤 사람이 신용카드 연체 횟수가 높을지 생각해보자 !

[시도해 볼 것]
- credit 별로 데이터를 나눠 각 신용도 점수별 특징을 자세히 보자 => 연체 횟수에 어떤 변수가 영향을 많이 줄 지 생각해보고 상관계수를 높이는 방향 
- binary, numerical 각각 변수 뜯어보기 
- 결측치 처리 방안 적용: 변화 있는지 보기 